In [2]:
# counts how many times a result appears in the data
def count_apperance(result_list,data):
    c = []
    for i in result_list:
        c.append((i,0))

    for i in range(len(c)):
        for j in range(len(data)):
            if c[i][0].issubset(data[j][1]):
                c[i] = (c[i][0],c[i][1]+1)
    return c

In [3]:
from itertools import combinations
# deletes all results where the subset is not in the result_list
def prune(k, result_list, new):
    for i in result_list:
        subsets = set(combinations(i,k))
        for j in subsets:
            j = set(j)
            if j not in new:
                result_list.remove(i)
                break
    return result_list

# generates possible combinations of the results
def self_join(k, c, support):
    new = [i[0] for i in c]
    result_list = []
    for i in range(len(new)):
        for j in range(i+1, len(new)):
                    temp1 = list(new[i])
                    temp2 = list(new[j])
                    if(temp1[0:k] == temp2[0:k]):
                        joined = set(temp1 + temp2)
                        result_list.append(joined)
                        
                        
    output = prune(k+1, result_list, new)
    return output

In [4]:
from itertools import combinations, product
import csv

# converts numbers to names fir output
def numberToName(c, names):
    output = []
    for i in range(len(c)):          
        new = set()
        for number in c[i][0]:
            new.add(names[number])
        output.append((new,c[i][1]))
    return output

# generates the output in the wanted format
def confidenceToName(confidence, names):
    output = []
    for i in range(len(confidence)):          
        old = set()
        new = set()
        for number in confidence[i][0]:
            old.add(names[number])
        for number in confidence[i][1]:
            new.add(names[number])
        output.append((old,new,confidence[i][2], confidence[i][3], confidence[i][4]))
    return output

# calculates the confidence and saves it
def getConfidence(old, new):
    output=[]
    for i in old:
        for j in new:
            if i[0].issubset(j[0]):
                oldsup = i[1]
                newsup = j[1]
                conf = newsup/oldsup
                output.append((i[0],j[0]-i[0],conf, oldsup,newsup))
    return output

In [5]:
# aprori algorithm mostly consists out of other functions
def apriori(database, products, data, support, conf):    # first round to initialize c
    support = support*len(data)
    k = 1
    c = []
    for i in range(len(database[0])):
        c.append(({i},0))

    for i in range(len(c)):
        for j in range(len(data)):
            if c[i][0].issubset(data[j][1]):
                c[i] = (c[i][0],c[i][1]+1)


    k = 0
    # c is previous list with k = x counted is new list with k = x+1. before end c becomes only good relations
    while c:
        result_list = self_join(k,c, support)
        counted = count_apperance(result_list, data)
        counted = [item for item in counted if item[1] >= support]
        
        
        confidence = getConfidence(c, counted)
        confidence = [item for item in confidence if item[2] >= conf]
        
        
        c = counted
        if c:
            names = confidenceToName(confidence, products)
            print("layer", k+1, "total:", len(confidence))
            for i in names:
                print(i[0], i[3], "==>", i[1], i[4], "confidence:", i[2] )
        k+=1
    return names


In [6]:
file_path = 'aap.txt'  # Replace 'your_file.txt' with the path to your text file

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read all lines from the file
    lines = file.readlines()

# Extract names from the lines, skipping the index at the beginning
products = [line.split()[1].strip() for line in lines]


database = []
with open('myDataFile.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        database.append(row)

data = []
for i in range(len(database)):
    data.append((i,set()))

# makes the database to workable format
for i in range(len(database)):
    for j in range(len(database[i])):
        if database[i][j] == 't':
            data[i][1].add(j) 
output = apriori(database, products, data, 0.005, 0.6)

layer 1 total: 0
layer 2 total: 13
{'bottled_water', 'butter'} 88 ==> {'whole_milk'} 53 confidence: 0.6022727272727273
{'domestic_eggs', 'butter'} 95 ==> {'whole_milk'} 59 confidence: 0.6210526315789474
{'root_vegetables', 'butter'} 127 ==> {'whole_milk'} 81 confidence: 0.6377952755905512
{'tropical_fruit', 'butter'} 98 ==> {'whole_milk'} 61 confidence: 0.6224489795918368
{'whipped_sour_cream', 'butter'} 100 ==> {'whole_milk'} 66 confidence: 0.66
{'yogurt', 'butter'} 144 ==> {'whole_milk'} 92 confidence: 0.6388888888888888
{'tropical_fruit', 'curd'} 101 ==> {'whole_milk'} 64 confidence: 0.6336633663366337
{'domestic_eggs', 'margarine'} 82 ==> {'whole_milk'} 51 confidence: 0.6219512195121951
{'domestic_eggs', 'pip_fruit'} 85 ==> {'whole_milk'} 53 confidence: 0.6235294117647059
{'domestic_eggs', 'tropical_fruit'} 112 ==> {'whole_milk'} 68 confidence: 0.6071428571428571
{'root_vegetables', 'onions'} 93 ==> {'other_vegetables'} 56 confidence: 0.6021505376344086
{'pip_fruit', 'whipped_sour_